<a href="https://colab.research.google.com/github/cedro3/others2/blob/main/dlfs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# セットアップ


In [ ]:
# githubのコードをコピー
!git clone https://github.com/SenHe/DLFS.git
%cd DLFS/

# ライブラリーのインストール
!pip3 install -r requirements.txt

# 補助モデルのダウンロード
!python download_models.py

# 学習済みモデルのダウンロード
! pip install --upgrade gdown
import gdown
!mkdir checkpoints
%cd checkpoints

gdown.download('https://drive.google.com/u/0/uc?id=1pB4mufFtzbJSxxv_2iFrBPD3vp_Ef-n3&export=download', 'males_model.zip', quiet=False)
!unzip males_model.zip
gdown.download('https://drive.google.com/u/0/uc?id=1z0s_j3Khs7-352bMvz8RSnrR53vvdbiI&export=download', 'females_model.zip', quiet=False)
!unzip females_model.zip
%cd ..

# サンプル画像ダウンロード
gdown.download('https://drive.google.com/uc?id=1ruwDizjnzd3scR1QvpXGWLXywY8_W0yj', './images.zip', quiet=False)
!unzip images.zip

# インポート＆初期設定

In [ ]:
# インポート＆初期設定
import os
from collections import OrderedDict
from options.test_options import TestOptions
from data.data_loader import CreateDataLoader
from models_distan.models import create_model
import util.util as util
from util.visualizer import Visualizer

opt = TestOptions().parse(save=False)
opt.display_id = 0 # do not launch visdom
opt.nThreads = 1   # test code only supports nThreads = 1
opt.batchSize = 1  # test code only supports batchSize = 1
opt.serial_batches = True  # no shuffle
opt.no_flip = True  # no flip
opt.in_the_wild = True # This triggers preprocessing of in the wild images in the dataloader
opt.traverse = True # This tells the model to traverse the latent space between anchor classes
opt.interp_step = 0.05 # this controls the number of images to interpolate between anchor classes

data_loader = CreateDataLoader(opt)
dataset = data_loader.load_data()
visualizer = Visualizer(opt)


# 年齢による顔アニメーション

In [ ]:
# 年齢による顔アニメーション作成
opt.name = 'females_model' # females_model'あるいは'males_model'を選択する
model = create_model(opt)
model.eval()

img_dir ='images'  # フォルダー指定
img_file ='04.jpg'  # ファイル名
img_path = img_dir+'/'+img_file
data = dataset.dataset.get_item_from_path(img_path)
visuals = model.inference(data)

os.makedirs('results', exist_ok=True)
out_path ='results/'+img_file[:-4]+'.mp4'
visualizer.make_video(visuals, out_path)

# コーデック変換
import os
import shutil
#shutil.copy('./results/'+img_path[:-4]+'.mp4', './results/out.mp4')
shutil.copy(out_path, './results/out.mp4')
if os.path.exists('./output.mp4'):
   os.remove('./output.mp4')
! ffmpeg -i ./results/out.mp4 -vcodec h264 -pix_fmt yuv420p output.mp4

In [ ]:
# mp4動画の再生
from IPython.display import HTML
from base64 import b64encode

mp4 = open('./output.mp4', 'rb').read()
data_url = 'data:video/mp4;base64,' + b64encode(mp4).decode()
HTML(f"""
<video width="50%" height="50%" controls>
      <source src="{data_url}" type="video/mp4">
</video>""")